In [97]:
import os
from tqdm import tqdm

from scipy.io import wavfile
from scipy import signal

import numpy as np

import pandas as pd
from python_speech_features import mfcc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
dataset_path = 'dataset/TorontoDataset/'
folders = os.listdir(dataset_path)
categories = {i:folders[i] for i in range(len(folders))}
categories.items()

dict_items([(0, 'Sad'), (1, 'Neutral'), (2, 'Pleasant Surprise'), (3, 'Happy'), (4, 'Disgust'), (5, 'Angry'), (6, 'Fear')])

In [37]:
def features_extraction(sig, rate, nfft=2500, **kwargs):
    mfcc_feat = mfcc(sig, rate, nfft=nfft, **kwargs)
    mfcc_mean = mfcc_feat.mean(axis=0)
    mfcc_std = mfcc_feat.std(axis=0)
    return mfcc_mean, mfcc_std

In [38]:
dataset = np.zeros((0,28))
errors = ''
for y, category in categories.items():
    waves = [file for file in os.listdir(dataset_path+category) if file.endswith('wav')]
    for wav in tqdm(waves,desc=category):
        path = os.path.join(dataset_path,category,wav)
        try:
            rate, sig = wavfile.read(path)
        except ValueError:
            errors += 'Error processing '+path+'\n' 
            continue
        mfcc_mean, mfcc_std = features_extraction(sig, rate)
        features = np.concatenate(([path],mfcc_mean, mfcc_std, [y])).reshape(1,28)
        dataset = np.concatenate((dataset,features))
print(errors)

Fear: 100%|██████████| 400/400 [00:13<00:00, 29.84it/s]

Error processing dataset/TorontoDataset/Angry/YAF_germ_angry.wav
Error processing dataset/TorontoDataset/Fear/YAF_neat_fear.wav



In [47]:
dataset = dataset.astype('<U5')

In [49]:
np.savetxt('toronto.csv',dataset,delimiter=',', fmt="%s")

In [50]:
import xgboost as xgb

In [66]:
df = pd.read_csv('toronto.csv',header=None)
df = df.drop(columns=[0])

,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,26,27
0,15.87,-2.040,12.310,12.830,-10.000,-9.380,-1.100,-15.900,-6.94,-4.100,...,18.03,18.53,12.580,18.75,9.364,13.54,9.633,11.120,9.566,0
1,15.58,-8.700,12.170,8.205,-3.230,-11.700,-2.810,-21.200,-3.79,-3.240,...,14.10,14.87,12.750,15.24,11.530,11.22,11.290,10.140,7.980,0
2,12.37,-4.730,11.540,-0.180,3.985,2.337,-7.810,-9.260,-14.40,4.614,...,13.63,13.21,20.100,17.23,17.300,13.00,14.470,10.730,11.230,0
3,12.95,-1.390,16.400,-1.060,2.741,0.700,-6.890,-7.840,-17.50,-1.660,...,11.90,11.22,17.660,14.26,16.150,13.76,9.894,11.300,10.490,0
4,15.39,-4.420,13.620,7.700,-13.300,-6.420,3.233,-16.800,-2.95,-6.200,...,14.59,20.62,15.730,21.37,9.921,13.45,9.677,7.920,9.528,0
5,15.38,-6.580,11.120,6.541,-6.220,-6.360,-0.030,-16.300,-4.64,-2.360,...,15.41,17.61,14.580,18.01,11.590,12.63,11.740,9.054,10.430,0
6,12.35,-1.810,17.310,4.614,4.915,-0.080,-10.700,-4.080,-15.40,-4.930,...,12.21,11.10,18.330,16.89,14.490,15.54,11.610,9.792,7.843,0
7,16.05,-7.790,9.038,8.494,-5.580,-11.100,-2.490,-15.400,-5.20,3.326,...,12.38,20.06,10.250,21.45,9.878,14.54,11.350,8.573,9.738,0
8,15.92,-5.260,8.241,13.290,-8.020,-13.900,-3.670,-15.900,-5.45,1.405,...,14.44,17.83,11.440,21.20,10.230,12.47,10.260,11.070,7.565,0
9,13.54,-5.810,14.330,1.925,7.293,-3.160,-13.100,-9.790,-21.50,3.460,...,14.46,11.00,23.170,14.73,16.850,13.87,9.838,11.020,10.010,0


In [88]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [99]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train,y_train)
predictions = gbm.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.81%


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [101]:
results = pd.DataFrame()
results['Predicted'] = [categories[val] for val in predictions]
results['Actual'] = [categories[val] for val in y_test]
results.head(5)

,Predicted,Actual
0,Sad,Sad
1,Disgust,Disgust
2,Angry,Angry
3,Disgust,Disgust
4,Neutral,Neutral


In [102]:
results.loc[df['Predicted'] != df['Actual']]

,Predicted,Actual
11,Neutral,Sad
19,Pleasant Surprise,Disgust
48,Disgust,Pleasant Surprise
104,Disgust,Neutral
105,Happy,Fear
116,Neutral,Sad
130,Happy,Sad
148,Neutral,Sad
173,Happy,Pleasant Surprise
177,Happy,Angry


In [107]:
from sklearn.model_selection import cross_val_score

In [110]:
scores_cross = cross_val_score(gbm,X,y,cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [112]:
scores_cross

array([0.96607143, 0.94821429, 0.94821429, 0.96071429, 0.95878136])